---
# Basics of Mobile Robotics Project

## Index

## Introduction

# Image Processing



> <img src="./imgs/pdf-based_lacation_estimate.png" alt="pdf-based_lacation_estimate" style="width: 400px;"/>

> *Here is an example of the image we are working with :*

# The image processing can be divided into four steps :

### 1) Obtain an image of the region of interest : the map of the envirronment.
We want to get rid off all the random components introduced by taking a picture. it's include translations, rotations, scaling and most importantly perspective transformations. Therefore we want to search four points, which we willingly place at known distances and angles from each other, to standardize our image. 

The best is to detect the four extreme angles of the rectangle environment. It's increases the precision of the transformation by decreasing the relative error on their distance to each other. There are several ways to find these corner : 

- detect the lines of the rectangle (hough transform or edge detection) but there will be to many line with obstacles. It could have been possible to detect the edges of the outer rectangle defining the map, fill it (meaning put all the pixels inside its border to one), next erode the image to delete all exterieur components to the rectangle, our region of interest. Then we just have to dilate again to have only a big rectangle of white pixels on a black background, which is very easy to detect. But our backround had too much long lines throughout the whole image (for example foots of the table) then filling closed borders would have lead to fill the entire image. To summarize, this is not a good method because our region of interest does not produce the biggest closed border.
- convolve the image with a corner (two small lines 90 degrees to each others) has the advantage of being quite scale invariant but depends on rotation. The results were very bad as the image often presents orientations. Furthemore with image with grid pattern we can't distinguish all the corners.
- convolve with a disk has the advantage of being rotation invariant, which is better than scale invariant. So we introduced specific patterns at the corners to inderictly detect them. Its better to have disk with specific color, but even then convolution was not good enough (with perspective, disk become ellipsoids and are not detected at all).
-use the HSV (Hue Saturation Value) representation of the image to easily detect colors. We pass from RGB representation where it's hard to detect specific colors to the HSV where saturated colors (meaning very red, very blue or very green) are easy to detect on a neutral backgound. Therefore we just need colors, without specific shape. We kept the disks because they are more robust to bad detection : when some pixels are not detect they mostly stay as one connected region of pixels with the same centroid.
We chose the last option, the most efficient.



0) just display the orginal image.

1) Obtain an image of the region of interest : the map of the envirronment.

2) Find the position of the robot.

3) Find the goal

4) get a map for the global path planning




Here we import libraries and define usefull functions and variables

In [ ]:
#!pip install scikit-image

In [ ]:
import cv2
import numpy as np
import math
from skimage import morphology
import matplotlib.pyplot as plt
from matplotlib import colors
np.random.seed(0) # To guarantee the same outcome on all computers


#variable definition
corner= np.zeros((4,2),np.uint32)  #matrix of the detected patterns' centers. int because thee are index
corner_sort = np.zeros((4,2),np.uint32)  #sorted matrix of the detected patterns' centers
distancex = 1200  #true distance of the robot environment
distancey =  800  #true distance of the robot environment
start = np.zeros(3,np.float32) #starting point of ther robot (x,y,angle). float to compute the angle
end = np.zeros(2,np.int32) #goal point (x,y)
centroid = np.zeros((2,3),np.int32) #centroids of the red disk and their number of points
robot_size_kernel = (15,15)#kernel of the size of the robot in pixel in the resized map
capt_point = (850,700)#point at which we emulate the sensing of an onboard camera
capt_xdim = 200#dimension of the simulated camera view
capt_ydim = 200
img_path = 'photos_test_3\IMG_5939.JPG'#if we take the image from the computer

goalReached = -100#is goal reached? terminate the program
capt_xdim = 200 #dimension of the emulated view, the bigger it is, the more the precision of the postion is
capt_ydim = 200
true_robot_length = 20#length between the center of the robot and a bit forward in pixels in the true map




#function definition


##################################################################################################
#Image processing functions__START
##################################################################################################
def stackImages(scale, imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range(0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape[:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]),
                                                None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y] = cv2.cvtColor(imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank] * rows
        hor_con = [imageBlank] * rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None, scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor = np.hstack(imgArray)
        ver = hor
    return ver

def display(output, fact = 0.3):
    cv2.namedWindow("output", cv2.WINDOW_NORMAL )  # Create window with freedom of dimensions
    cv2.resizeWindow("output", math.floor ( fact * output.shape [1] ) , math.floor (fact * output.shape [0] ) )  # Resize window to specified dimensions keeping the image form
    cv2.imshow("output", output)
    cv2.waitKey ( 0 )
    
def angle(x0,y0,x1,y1):
    #directly put the angle in the start array
    dist = math.sqrt ( (x0 - x1)*(x0 - x1) + (y0 - y1)*(y0 - y1)  )
    x = (x1 - x0)/dist#normalisation required
    y = (y1 - y0)/dist
    start[2] = math.acos(x)
    
def prepare_image(path):
    #read the image saved in path and return it in RGB mode
    img = cv2.imread(path)
    u,v,d = img.shape
    if (u>v):#if needed rotate the image to have a horizontal one
        img = cv2.rotate(img,cv2.ROTATE_90_CLOCKWISE)
    imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    return img

def get_clean_map(img) :
    #find the blue corner, make them wite, and map the image to get a nice map
    #return true_map, the map in high resolution

    #mask to get the blue color
    imgHSV = cv2.cvtColor ( img , cv2.COLOR_BGR2HSV )
    [h_min, s_min, v_min] = [90,138,108]
    [h_max, s_max, v_max] = [130,255,255]
    lower = np.array([h_min, s_min, v_min])
    upper = np.array([h_max, s_max, v_max])
    mask = cv2.inRange(imgHSV, lower, upper)

    #for robustness and avoid useless labeling, remove all the noisy blue isolated pixels
    img_dilate = cv2.dilate ( mask , np.ones ( (10 , 10) , np.uint8 ) , iterations=1 )
    img_erode = cv2.erode(img_dilate,np.ones((10,10),np.uint8),iterations = 2)
    img_dilate = cv2.dilate(img_erode,np.ones((10,10),np.uint8),iterations = 1)

    #put the same number to all connected pixels:
    labels = morphology.label(img_dilate, background=0)
    #seek for the four biggest blobs i.e. the four labels with the most of pixels
    #It's a robustness mesure : the four corners are supposed to have the same size and being much bigger than not wanted regions
    unique_labels = (np.unique(labels))[1:]
    comptage = np.zeros(unique_labels.size+1)
    for i in unique_labels :
        comptage[i] = np.sum(labels==i)
    indice = np.argsort(comptage)
    error =0
    if (np.max(indice)<4):
        return img
    for i in range(0,4):#take the four biggest labels
        ett_current = np.where(labels==(indice[indice.size-1-i]),1,0)
        #erase the blue pixels not to detect them again
        img [: , : , 0] = np.where ( labels == (indice [indice.size - 1 - i]) , 200 , img [: , : , 0] )
        img [: , : , 1] = np.where ( labels == (indice [indice.size - 1 - i]) , 200 , img [: , : , 1] )
        img [: , : , 2] = np.where ( labels == (indice [indice.size - 1 - i]) , 200 , img [: , : , 2] )
        #compute the centroids of each blob, they are the corner of the map
        ett_current = np.float32(ett_current)
        M = cv2.moments (ett_current)
        cX =  int ( M ["m10"] / M ["m00"] )
        cY =  int ( M ["m01"] / M ["m00"] )
        corner[i , :] = np.array([cX,cY])

    #sort the corner
    x_max = max(corner[:,0])
    x_min = min(corner[:,0])
    y_max = max(corner[:,1])
    y_min = min(corner[:,1])
    x_middle = (x_max-x_min)/2+x_min
    y_middle = (y_max-y_min)/2+y_min
    for i in range(0,4):
        if (corner[i,0]<x_middle) & (corner[i,1]<y_middle):
            corner_sort[0,:]=corner[i,:]
        if (corner [i , 0] < x_middle) &  (corner [i , 1] > y_middle) :
            corner_sort [1 , :] = corner [i , :]
        if (corner [i , 0] > x_middle) &  (corner [i , 1] < y_middle) :
            corner_sort [2 , :] = corner [i , :]
        if (corner [i , 0] > x_middle) &  (corner [i , 1] > y_middle) :
            corner_sort [3 , :] = corner [i , :]
    #apply resizing
    corner_goal = np.array([[0,0],[0,distancey], [distancex,0], [distancex,distancey]], np.float32)
    M = cv2.getPerspectiveTransform(np.float32(corner_sort),corner_goal)
    img_reconstruct = cv2.warpPerspective(img, M,(distancex,distancey))
    true_map = img_reconstruct
    return true_map

def robot_detection(map):
    #return the position of the robot (x,y,angle) angle between the direction where the robot is going and the lower abscisse axis

    #filter red component from the map
    imgHSV2 = cv2.cvtColor ( map, cv2.COLOR_BGR2HSV )
    [h_min , s_min , v_min] = [0 , 160 , 0]
    [h_max , s_max , v_max] = [180 , 255 , 255]
    lower = np.array ( [h_min , s_min , v_min] )
    upper = np.array ( [h_max , s_max , v_max] )
    mask = cv2.inRange ( imgHSV2 , lower , upper )

    # detect the big and the small blob
    img_erode = cv2.erode ( mask , np.ones ( (10 , 10) , np.uint8 ) , iterations=1 )
    img_dilate = cv2.dilate ( img_erode , np.ones ( (10 , 10) , np.uint8 ) , iterations=1 )
    labels = morphology.label ( img_dilate , background=0 )
    unique_labels = (np.unique ( labels )) [1 :]
    comptage = np.zeros ( unique_labels.size + 1 )
    for i in unique_labels :
        comptage [i] = np.sum ( labels == i )
    indice = np.argsort ( comptage )
    
    #plt.imshow(img_dilate)
    
    for i in range ( 0 , 2 ) :  # take the 2 biggest labels
        ett_current = np.where ( labels == (indice [indice.size - 1 - i]) , 1 , 0 )
        ett_current = np.float32 ( ett_current )
        sum = np.sum ( ett_current )
        M = cv2.moments ( ett_current )
        cX = int ( M ["m10"] / M ["m00"] )
        cY = int ( M ["m01"] / M ["m00"] )
        centroid [i , :] = np.array ( [cX , cY , sum] )
    #The biggest blob give the position (x,y) of the robot, we use also the small one to compute the angle
    imax = np.argmax ( centroid [: , 2] )
    imin = 1 - imax
    start [0 :2] = centroid [imax , 0 :2]
    angle ( centroid [imax, 0], centroid [imax, 1] , centroid [ imin, 0] ,centroid [imin,1]  )
    
    # Fix 3D issues & transform coords
    start[0] = start[0] - 21/400*(start[0] - distancex/2)   # correct for non-verticaltiy: estimated empirical values
    start[1] = distancey - start[1]                         # flip y dimension
    start[1] = start[1] - 31/600*(start[1] + 200)           # even more guesswork
    
    return start

def goal_detection(img_reconstruct):
    # detect the goal point

    #filter the green component
    imgHSV3 = cv2.cvtColor ( img_reconstruct , cv2.COLOR_BGR2HSV )
    [h_min , s_min , v_min] = [47 , 89 , 51]
    [h_max , s_max , v_max] = [116 , 209 , 198]
    lower = np.array ( [h_min , s_min , v_min] )
    upper = np.array ( [h_max , s_max , v_max] )
    mask = cv2.inRange ( imgHSV3 , lower , upper )


    #Here we consider there are not to much not wanted pixel in the mask
    #so we consider after erosion the only pixels to one are those from the goal
    img_erode = cv2.erode ( mask , np.ones ( (15 , 15) , np.uint8 ) , iterations=1 )
    img_dilate = cv2.dilate ( img_erode , np.ones ( (15 , 15) , np.uint8 ) , iterations=1 )

    M = cv2.moments ( img_dilate )
    cX = int ( M ["m10"] / M ["m00"] )
    cY = int ( M ["m01"] / M ["m00"] )
    end = np.array ( [cX , cY] )
    
    #fix coords
    end[1] = distancey - end[1]
    
    return end

def get_global_search_map(for_global_search):
    #return the map (meaning contains only obstacles) dilated to make obstacles a bit bigger
    #to take into account the size of the robot

    #here we get ride of all thin black lines used for online positioning
    map_gray = cv2.cvtColor ( for_global_search , cv2.COLOR_BGR2GRAY )
    map_dilate = cv2.dilate ( map_gray , np.ones ( (4 , 4) , np.uint8 ) , iterations=1 )
    map_erode = cv2.erode ( map_dilate , np.ones ( (4 , 4) , np.uint8 ) , iterations=1 )

    ret , map_bin = cv2.threshold ( map_erode , 50 , 255 , cv2.THRESH_BINARY )#apply a fixed threshold binarization
    map_bin = 255 - map_bin
    #increase the size of the obstacles
    map_global_search = cv2.dilate ( map_bin , np.ones ( robot_size_kernel , np.uint8 ) , iterations=1 )
    return map_global_search

def camera_emulation(true_map) :
    # emulate onboard camera return the emulated view, a rectangle a bit in front of the detected robot position
    # detect robot's position

    pos = robot_detection ( true_map )  # detect the position of the robot on the map

    # crop the image to have a small rectangle in front of the robot
    capt_point = np.array ( [pos [1] - capt_ydim , pos [0]] ) + true_robot_length * np.array (
        [-math.sin ( pos [2] ) , +math.cos ( pos [2] )] )  # the bottom left point of the emulated view
    capt_point = np.int32 ( capt_point )
    capt_rect = true_map [capt_point [0] : capt_point [0] + capt_xdim , capt_point [1] : capt_point [1] + capt_ydim]
    # display the filmed region by highlighting it
    true_map [capt_point [0] : capt_point [0] + capt_xdim , capt_point [1] : capt_point [1] + capt_ydim] = true_map [capt_point [0] : capt_point [0] + capt_xdim , capt_point [1] :capt_point [1] + capt_ydim] + 40

    # get the probable position of the robot by convolving the emulated view with the map
    res = cv2.matchTemplate ( true_map , capt_rect , eval ( 'cv2.TM_CCOEFF_NORMED' ) )
    min_val , max_val , min_loc , max_loc = cv2.minMaxLoc ( res )

    # find the center of the robot from max_loc, top left point of convolution template
    estimated_posx = max_loc [0] - math.floor ( true_robot_length * math.cos ( pos [2] ) )
    estimated_posy = max_loc [1] + capt_ydim + math.floor ( true_robot_length * math.sin ( pos [2] ) )
    cv2.rectangle(true_map, (estimated_posx , estimated_posy), (estimated_posx +10, estimated_posy+10), 255,2)

    return np.array ( [estimated_posx , estimated_posy] )
##################################################################################################
#Image processing functions___END
##################################################################################################
    

In [ ]:
#load img from computer
#img = prepare_image(img_path)#We load the image, we rotate it if needed to have a horizontal image.
#or get it from the webcam
#cap = cv2.VideoCapture(1)#put 1 if an external webcam is used
#cap.set(3,1200)#width
#cap.set(4,1000)#height
#success, img = cap.read()
#display(img)


#true_map = get_clean_map(img)
#for_global_search = cv2.resize(true_map, (100,50))#apply resizing to reduce computation cost of global search
#start = robot_detection(for_global_search)#starting point for global search
#end = goal_detection(for_global_search)#ending point for global search
#map_global_search = get_global_search_map(for_global_search)
#map_bin = map_global_search
#print('start :',start)
#print('goal :', end)
#display(for_global_search)

# Global Map

## Different Approaches
Different approaches exist to find an optimal global path. In the beginning a visibility graph was considered. To find the shortest path from start to goal Dijkstra’s algorithm was used. Due to difficulties in detecting the corners of the objects a new approach had to be found. 
Therefore a cell decomposition/grid based approach was implemented.

### A* on grid array
To find the optimal path from the start to end position, the A* algorithm is used on a binary grid array. The A* algorithm can be seen as an extension of Dijkstra's algorithm. The grid array represents free spaces with a zero and obstacles (which the robot has to navigate around) with a one. For the implementation, a modified version of the code from the Exercise session 5 of the course MICRO-452  “Basics of Mobile Robotics” was used:

### Testing Dijkstra - grid array
Following function allow the testing of the Dijkstra's allgorithm on a binary grid array, without needing all the setup (robot and surrounding).

In [ ]:
import os
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors


# Adding the src folder in the current directory as it contains the necessary scripts
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))
import PathPlanning

#max_val_column = 200 # Size of the map
#max_val_row=100
#occupancy_grid, cmap=PathPlanning.create_random_occupancy_grid(max_val_column, max_val_row)
#print(occupancy_grid.shape)

In [ ]:
# Define start and end goal -> values have to be changed, that the are on empty spot and that a path between them exists
#test_start=(0,0)#value has to be changed
#test_goal=(99,199)#value has to be changed
#path=PathPlanning.generate_global_path(test_start, test_goal, occupancy_grid)
#path=np.fliplr(path)
#print(f"path (form start to goal)")
#print(f"x={path[1]}")
#print(f"y={path[0]}")

# Control

Once the path planning has been done, we need to get the Thymio robot to follow the planned path.
We do so by implementing a pilot, which uses knowledge of the robot's location relative to its next goal to calculate the motor inputs necessary to reach this goal. 

Unfortunately, the robot does not always have perfect knowledge of its position at all times, so it must be estimated from the limited measurements it does have access to, namely
- motor odometry
- ground IR sensors 
- horizontal proximity sensors

Because both the movement process and these measurements are quite noisy, simply dead reckoning the robot's position is not accurate enough. We need to implement a stochasitc state estimator to try to estimate the robot's position from the noisy measurements.

> <img src="./imgs/control-Page-1.png" />

> *Figure 1: Control diagram of the robot and pilot*

## Estimator
A number of stochastic estimation methods exist. The approach chosen in this project, the Kalman Filter, is computationally one of the simplest. It was chosen in the hopes of making the system more reactive. The Kalman filter is normally used for systems with linear behavior, but it can relatively easily be extended to non-linear systems that do not vary too abruptly:
$$ \begin{eqnarray}
\bar{\mathbf x} &=& g(\hat{\mathbf x}, \mathbf u)\\
\bar{s} &=& G_{\mathbf x,\mathbf u}\cdot s\cdot G_{\mathbf x,\mathbf u}^T\\
K       &=& \bar{s}\cdot C^T\cdot (C\cdot\bar{s}\cdot C^T + R)^{-1}\\
\hat{\mathbf x} &=& \bar{\mathbf x} + K(\mathbf y-C\bar{\mathbf x})\\
s       &=& (I - K C)\cdot \bar{s}
\end{eqnarray}$$

## Control Signal
The Thymio robot may be controlled throught its two motors. Our control signal $\mathbf u$ will therefore be : 
$$ \mathbf u = \begin{bmatrix} u_r\\ u_l \end{bmatrix}$$

## State 
The state of the robot $\mathbf x$ is defined as follows :
$$ \mathbf{x} = \begin{bmatrix}x\\ y \\ \phi \\ v_r \\ v_l\end{bmatrix}$$
where $[x , y]^T$ are the robot's position in space, $\phi$ is its angular orientation with respect to the x-axis, and $[v_r , v_l]^T$ are the speed on the right and left wheels, respectively. 

## Motion Model
To apply the extended Kalman Filter, we need to define a motion model $\mathbf x^{+} = g(\mathbf{x,u})$. 
The simplest motion model for a robot such as the Thymio can be calculated in two steps. First, the intermediary values describing the change in angle and in distance are calculated:  
$$ \begin{bmatrix}dl\\ d\phi\end{bmatrix} = B\cdot \mathbf u$$ 

for $B=c \cdot T_s\begin{pmatrix} 0.5 & 0.5 \\  \frac{1}{d} & -\frac{1}{d} \end{pmatrix}$, where $T_s$ is the sample time and $d$ is the distance between the wheels. $c$ is a conversion constant to convert between Thymio speed and $mm/s$. 
Then the actual states may then be approximated as:
$$\begin{eqnarray}
\phi^{+} &=& \phi + d\phi\\
x^{+}    &=& x + dl \cdot cos(\phi^{+})\\
y^{+}    &=& y + dl \cdot sin(\phi^{+})\\
v_r^{+}  &=& u_r\\
v_l^{+}  &=& u_l
\end{eqnarray}$$

In order to linearize the motion model for use in the Kalman filter, we must calculate its Jacobian: $$G_{\mathbf{x,u}} = \begin{bmatrix} 
\frac{\partial g}{\partial x}(\mathbf{x,u}) &
\frac{\partial g}{\partial y}(\mathbf{x,u}) & 
\frac{\partial g}{\partial \phi}(\mathbf{x,u}) & 
\frac{\partial g}{\partial v_r}(\mathbf{x,u}) & 
\frac{\partial g}{\partial v_l}(\mathbf{x,u}) 
\end{bmatrix}$$

This can be done analytically, but it is too tedious for this report. The result can be found in the code in the file Robot.py.

Finally, we must estimate the uncertainty added by an update step of the motion process. We can measure the uncertainty on the motion process by wheel, which gives us $\mathbf{q} = \begin{bmatrix} q_r \\ q_l \end{bmatrix}$ resulting in a coviariance matrix $Q_{\mathbf u}$ with diagonal $\mathbf{q}$.
We can then apply the propagation of covariance formula though the linearized system (the Jacobian $G_{\mathbf{x,u}}$) in order to estimate the uncertainty on the robot state gained due to noise for a single process step:
$$ Q = G_{\mathbf{x,u}} \begin{pmatrix} \mathbf 0 & 0 \\ 0& Q_{\mathbf{u}}\end{pmatrix} G_{\mathbf {x,u}}^T$$
**Note:** This is in fact illegal. The real probablity density of the error added to the robot's state is highly non-linear. It follows a banana-shape, as the anguar uncertainty is much larger than the distance uncertainty. However, it was estimate(d) that the robot would have ample chance to localize itself and thus reduce the uncertainty to a level where the non-linearity doesn't matter as much.


## Sensor interpretation 

As mentioned above, the robot has access to several sensors which give it information about the world surrounding it. 
In our application, the sensors available to us do not allow a reversible mapping into state-space (i.e: the sensor function $\mathbf y = h(\mathbf x)$ is non-invertible). This is because the map is made up of a repeating grid, so we have no information on the absolute localization, and furthermore from the pont of view of the sensor, there is no way to tell whether the detected line is horizontal or vertical.
We must therefore do a preprocessing step on the raw sensor values to put them into a useable form for our (over)extended Kalman estimator.

A first approach was, whenever a line was detected by a sensor to simply take the coordinate of the closest gridline to the estimated position of that sensor as the "measured value" $\tilde{\mathbf y}$. This approach however does not take into account the fact that the estimated postion of the robot can be quite uncertain, so the returned "measured values" are very often wrong.

A different approach is to incorporate our estimated uncertainty into the sensor interpretation. Instead of looking for the closest gridline, we look for the most likely coordinates in which we could encounter a gridline, as shown in figure 3 below. These are then taken as $\tilde{\mathbf y}$.
This approach was adapted from : https://www.cs.cmu.edu/~motionplanning/papers/sbp_papers/integrated2/machler_grid_odometry.pdf



> <img src="./imgs/pdf-based_lacation_estimate.png" alt="pdf-based_lacation_estimate" style="width: 400px;"/>

> *Figure 2 : PDF-based location estimation. The real position of the sensor in space is unkown, so it is modelled as multivariate gaussian probability density function. When a gridline is encountered, we can calculate the position on the gridlines with the highest probablilty, and take this as our new position estimate.*

In [ ]:
import os
import sys
import time
import serial
import matplotlib.pylab as plt
import numpy as np
from threading import Timer
from IPython import display as ds

# Adding the src folder in the current directory as it contains the necessary scripts
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from Thymio import Thymio
import ReapeatedTimer
import Field
import Robot
import util

In [ ]:
def robot_thread(field, robot, goals, camera_pos):
    """ do the robot control cycle """

    if not goals :
            rt.stop()   #this is kind of bad, but i'm no python expert, so it'll have to do
            return 0

    u,local_obstacle = robot.pilot(goals)

    if not local_obstacle :
        robot.do_motion(u)

    sensor_data = robot.do_measure()
    
    if local_obstacle : 
        u = sensor_data[9:11,:]
        u[u>2**15] = u[u>2**15] - 2**16

    y = robot.sensor_interpretation(sensor_data, field)

    if camera_pos.x.size :                                    # if a camera position is available, update
        c0 = np.concatenate((np.eye(3), np.zeros((3,2))), axis = 1)
        y0 = camera_pos.x[0:3,:]
        r0 = np.array([1, 1, .01])**2                           # we assume the camera position is the most precise possible
        robot.R = np.diag(np.concatenate((r0, np.diag(robot.R)), axis=0))  
        robot.C = np.concatenate((c0, robot.C))
        y = np.concatenate((y0, y), axis=0)
        camera_pos.x = np.array([])

    xtemp, stemp = robot.kalman_estimator(u, y)

    robot.xodo = robot.motion_model(robot.xodo, u)             # make a purely odometry-based estimate, for visual comparison

    field.xhat  = np.concatenate((field.xhat, xtemp), axis=1)
    field.xodo  = np.concatenate((field.xodo, robot.xodo), axis=1)
    field.s.append(2*np.sqrt(stemp[0,0]+stemp[1,1]))  
        
        
def camera_thread(field, cap, camera_pos) : 
    """ do the camera position acquisition cycle """
    
    success, img = cap.read()                                  # read the image from the  fixed camera
    true_map     = get_clean_map(img)                          # get the clean in good resolution
    
    x = util.adapt_vision_coords(robot_detection(true_map), field)# detect the position of the robot on the map
    
    field.xreal = np.concatenate((field.xreal, x), axis=1)
    camera_pos.x = x.copy()

# Local Avoidance Protocol

# TEST

In [ ]:
robot      = Robot.NewRobot()                                                   # robot 
field      = Field.NewField(1200, 800, 200, 4)                                  # table

robot.th   = Thymio.serial(port="COM15", refreshing_rate=0.1)                   # set up Thymio communication

In [ ]:
# setup vision
cap_size          = (1200, 1000)
global_shape      = (90,  60)
cap               = cv2.VideoCapture(1)                                         # put 1 if an external webcam is used
cap.set(3,cap_size[0])                                                          # width
cap.set(4,cap_size[1])                                                          # height
success, img      = cap.read()
plt.imshow(img)
#display(img)

true_map          = get_clean_map(img)

start             = util.rescale_int_coords(robot_detection(true_map), (true_map.shape[1], true_map.shape[0]), global_shape)  # starting point for global search
end               = util.rescale_int_coords(goal_detection(true_map),  (true_map.shape[1], true_map.shape[0]), global_shape)  # end point for global search
map_global_search = get_global_search_map(cv2.resize(true_map, global_shape))# apply resizing to reduce computation cost of global search

# setup global path finding
occupancy_grid    = np.flipud(np.asarray(map_global_search))                     # make occupancy grid and change coordinate system           
#occupancy_grid, _ = create_random_occupancy_grid(90,60)

#path              = generate_global_path((occupancy_grid.shape[0]-start[1], start[0]), (occupancy_grid.shape[0]-end[1], end[0]), occupancy_grid)
path              = generate_global_path((start[1], start[0]), (end[1], end[0]), occupancy_grid)
path              = util.adapt_pathfinder_coords(path, field, occupancy_grid.shape)



# setup robot 
Tr = .2                                                                          # robot update timestep
Tc =  1                                                                          # camera update timestep

startpos    = util.adapt_vision_coords(robot_detection(true_map), field)         # starting point for pilot
robot.xhat  = startpos.copy()                                                    # setup robot
robot.xodo  = startpos.copy()
robot.s     = np.diag([1, 1, .1, 20, 20])**2

robot.d     = 90
robot.set_Ts(Tr)
robot.maxv  = 10
robot.maxw  = np.pi/32

field.goals = path
field.xreal = startpos.copy()                                                    # prepare record of (mis)deeds
field.xhat  = robot.xhat.copy()
field.xodo  = robot.xodo.copy()
field.s     = [2*np.sqrt(robot.s[0,0]+robot.s[1,1])]

goals       = np.ndarray.tolist(field.goals)

class Camera_Pos :
    def __init__ (self): 
        pass
camera_pos = Camera_Pos()
camera_pos.x = np.array([])                                                      # communication variable

rt = RepeatedTimer.Thread(Tr, robot_thread,  field, robot, goals, camera_pos);   # start threads
ct = RepeatedTimer.Thread(Tc, camera_thread, field, cap, camera_pos);

try:
    #print("Starting Run")  
    for i in range(20):
        time.sleep(1) 
        plt.clf();
        plt.gcf().set_size_inches(12,8)
        plt.imshow(occupancy_grid,cmap=colors.ListedColormap(['white', 'black']), origin = 'lower', extent=(field.xmin, field.xmax, field.ymin, field.ymax))
        field.plot()
        ds.display(plt.gcf())
        ds.clear_output(wait=True)
        
        if not goals : 
            print("arrived")
            break

finally:
    rt.stop()
    ct.stop()
    robot.th.set_var("event.args", 2)
    robot.do_motion(np.zeros((2,1)))
    print("Done");

In [ ]:
ct.stop()

In [ ]:
robot.do_motion(np.zeros((2,1)))
robot.th.close()

In [ ]:
plt.imshow(occupancy_grid,cmap=colors.ListedColormap(['white', 'black']), origin = 'lower', extent=(field.xmin, field.xmax, field.ymin, field.ymax))
field.plot()
plt.gcf().set_size_inches(12,8)